In [ ]:
from tardis import run_tardis

#model_name = 'hit_right_lte_l300_150te_dilute-blackbody_8.3Lsun'
#model_name = 'hit_5.0e-11_300_300_1.0C0.0O_right13000_lte_dilute-lte_dilute-blackbody_8.2Lsun_3.5d'
model_name = 'exp2000_5.0e-12_0.5C0.5O_right13000_lte_dilute-lte_dilute-blackbody_8.6Lsun'
#model_name = 'M2023_0.1_1.0e-10_+0_0.5C0.5O_12000_lte_dilute-lte_dilute-blackbody_8.2Lsun_3.5d'
#model_name = 'DDC_0.5C0.5O_13000_lte_dilute-lte_dilute-blackbody_8.1Lsun_3.5d'
#model_name = 'test_nlte'

sim = run_tardis("SN2024igg.yml",
                 virtual_packet_logging=True,
                 show_convergence_plots=True,
                 export_convergence_plots=True,
                 log_level="INFO")

Auto-detected Jupyter notebook environment


Auto-detected Jupyter notebook environment


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

Initializing tabulator and plotly panel extensions for widgets to work


BokehModel(combine_events=True, render_bundle={'docs_json': {'8fe76a15-cc97-4bfa-8a44-8aa6376fd6b1': {'version…

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


    'data': [{'type': 'scatter', 'uid': '1514f4c7-89c0-4d56-81d5-03dd9b2f8c7a', …

In [ ]:
import matplotlib.pyplot as plt

spectrum = sim.spectrum_solver.spectrum_real_packets
#spectrum_virtual = sim.spectrum_solver.spectrum_virtual_packets
#spectrum_integrated = sim.spectrum_solver.spectrum_integrated

In [ ]:
#sim.to_hdf('tmp/hit_right_lte_lte_8.3Lsun.hdf',name='simulation')

In [ ]:
%matplotlib widget
import astropy.units as u
import numpy as np
plt.xlabel(r"Wavelength [$\AA$]")
plt.ylabel(r"Luminosity density [erg/s/cm$^2$/$\AA$]")
d = 51.18 * u.Mpc
wave_lambda = spectrum.wavelength.value
F_lambda = (spectrum.luminosity_density_lambda / (4*np.pi*d**2)).to(u.erg/u.s/u.cm**2/u.AA).value
print(wave_lambda)
print(F_lambda)
plt.plot(wave_lambda, F_lambda)
plt.show()
np.savetxt('%s.txt'%model_name, np.array([wave_lambda[::-1], F_lambda[::-1]]).T)

In [ ]:
#%matplotlib inline
%matplotlib widget
plt.figure(figsize=(10, 6.5))

spectrum.plot(label="Normal packets")
#spectrum_virtual.plot(label="Virtual packets")
#spectrum_integrated.plot(label='Formal integral')

plt.xlim(500, 9000)
plt.title("TARDIS example model spectrum")
plt.xlabel(r"Wavelength [$\AA$]")
plt.ylabel(r"Luminosity density [erg/s/$\AA$]")
plt.legend()
plt.show()

In [ ]:
from tardis.visualization import SDECPlotter
import astropy.units as u
import numpy as np

In [ ]:
plotter = SDECPlotter.from_simulation(sim)

In [ ]:
import extinction
data = np.loadtxt('2024igg_2024-05-07_00-00-00_NOT_ALFOSC_None.ascii')
mag_ext = extinction.ccm89(data[:,0], 3.1 * 0.0112, 3.1)
data[:,1] = data[:,1] * 10 ** (0.4 * mag_ext) * (1 + 0.01085)
'''
spec_mag = spec_convolution(data0[:,0], data0[:,1]*1e-17, 'r', filters_dict={'r':'Palomar_ZTF.r+AB'}) #17.129563376928594
phot_mag = (17.909065681158733-17.128310518183657)/(60437.262743-60438.259815)*(60437.999220-60438.259815)+17.128310518183657
factor = 10**(-0.4*(phot_mag-spec_mag))
'''
data[:,1] = data[:,1]*1e-17*0.8296
data[:,0] = data[:,0]/(1+0.01085)

observed_spectrum_wavelength, observed_spectrum_flux, observed_spectrum_flux_err = data.T
observed_spectrum_wavelength = observed_spectrum_wavelength * u.AA
observed_spectrum_flux = observed_spectrum_flux * u.erg / (u.s * u.cm**2 * u.AA)

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'     
plt.rcParams['mathtext.fontset'] = 'custom'         
plt.rcParams['mathtext.rm'] = 'Times New Roman'     
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'  
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'    


plt.rcParams['font.weight'] = 600
plt.rcParams['axes.labelweight'] = 600
plt.rcParams['axes.titleweight'] = 600
plt.rcParams['figure.titleweight'] = 600
plt.rcParams['figure.titleweight'] = 600

fig, ax = plt.subplots(figsize=(8,6))
ax = plotter.generate_plot_mpl('real', species_list=['C II', 'O I', 'He I'], observed_spectrum = (observed_spectrum_wavelength, observed_spectrum_flux), distance=51.18 * u.Mpc,
                              ax=ax,packet_wvl_range=np.array([3300,10000])*u.AA, blackbody_photosphere=False, cmapname='cool')

plt.rcParams["font.family"] = 'Times New Roman'
spec_M2023 = np.loadtxt('spect_031.d')
new_scale = 0.83/0.33
spec_M2023[:,1] = spec_M2023[:,1]*0.0021713347070782033*new_scale
mark = (spec_M2023[:, 0] > 3500)*(spec_M2023[:, 0] < 10000)
ax.plot(spec_M2023[:,0][mark], spec_M2023[:,1][mark], label='CSM_M2023', color='orange')

ax.plot(data[:,0], data[:,1], c='black')

ax.plot([3770, 3770], [3.5e-16*new_scale,5.5e-16*new_scale], color='black')
ax.plot([4103, 4103], [3.17e-16*new_scale,5.11e-16*new_scale], color='black')
ax.plot([4503, 4503], [2.77e-16*new_scale,4.64e-16*new_scale], color='black')
#ax.plot([5650, 5650], [2.087e-16*new_scale,3.3e-16*new_scale], color='black')
ax.plot([6270, 6270], [1.2e-16*new_scale,2.57e-16*new_scale], color='black')
ax.plot([6930, 6930], [9.8e-17*new_scale,1.8e-16*new_scale], color='black')
ax.plot([3770, 6930], [5.5e-16*new_scale,1.8e-16*new_scale], color='black')
ax.plot([5800, 5800], [3.12e-16*new_scale,3.62e-16*new_scale], color='black')
ax.text(5800, 3.65e-16*new_scale, 'CII', va='bottom', ha='center')

ax.plot([7500, 7500], [0.9e-16*new_scale, 1.5e-16*new_scale], c='black')
ax.text(7500, 1.55e-16*new_scale, 'OI', va='bottom', ha='center')

plt.rc('font', size=14)           
plt.rc('axes', titlesize=14)      
plt.rc('axes', labelsize=14)      
plt.rc('xtick', labelsize=14)     
plt.rc('ytick', labelsize=14)    
plt.rc('legend', fontsize=14)     

ax.set_xlabel('Rest Wavelength [$\\rm \\AA$]', fontsize=14)
ax.set_ylabel('Flux Density $f_{\\lambda}$ [$\\rm erg\\ s^{-1}\\ \\AA^{-1}\\ cm^{-2}$]', fontsize=14)

ax.text(5650, 2.087e-16*new_scale, '?', color='black', fontsize=20)

ax.legend()

fig.savefig(model_name + '.pdf', bbox_inches='tight')

